This script is used to visualize the data collected

In [ ]:
#Imoport packages

import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#Import BLE data and save t as a pandas dataframe

file_path = "/Users/pdmcguckian/Documents/IoT/Project/SIoT/Data & Modelling/BLEData.csv"
data_set = pd.read_csv(file_path)
dataset = pd.DataFrame(data_set)

#Fill any missing values with 0
dataset.fillna(0)

#Print detials of dataset
print("Dimension of dataset: ", dataset.shape)
tmp_str = "Feature(attribute)     DataType"; print(tmp_str+"\n"+"-"*len(tmp_str))
print(dataset.dtypes)

In [ ]:
#Remove the ID column given by default vy mongo
dataset = dataset.drop(['_id'], axis=1)

#Convert timestamps into human readable format
for i in range(len(dataset['timeStampID'])):
    try:
        unix = dataset['timeStampID'][i]
        timestr = time.ctime(unix)[11:19]
        dataset['timeStampID'][i] = timestr
    except:
        dataset['timeStampID'][i] = 0

#Display results
dataset.head()

In [ ]:
#Script to seperate dataset into specfic arrays for each days worth of BLE data

#Each array inside of out arrays contains on days worth of BLE data
times = [[]]
bdrms = [[]]
ktchns = [[]]
lvngrms = [[]]

#Loop through entire dataset and save values to array
day = 0
for index, row in dataset.iterrows():

    #If time is mid day, system resets to collect the next day
    if row['timeStampID'] == "12:00:00":
        day += 1
        print(row)
        times.append([])
        bdrms.append([])
        ktchns.append([])
        lvngrms.append([])
    
    #Add current values to array
    times[day].append(row['timeStampID'])
    bdrms[day].append(row['rssiBDRM'])
    ktchns[day].append(row['rssiKTCHN'])
    lvngrms[day].append(row['rssiLVNGRM'])

In [ ]:
#Script plots every day

for i in range(1, len(times)-1):
    fig, ax = plt.subplots()

    #Plots BLE data for eacg room on the same gragh for visdualuizatiion
    ax.plot(times[i][:1400], bdrms[i][:1400], linewidth=1.0, label = "Bedroom")
    ax.plot(times[i][:1400], lvngrms[i][:1400], linewidth=1.0, label = "Living Room")
    ax.plot(times[i][:1400], ktchns[i][:1400], linewidth=1.0, label = "Kitchen")
    
    #Only plots a few x values - otherwithse woulds be unreadabvle
    every_nth = 240
    for n, label in enumerate(ax.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)

    plt.show()

In [ ]:
#Script converts plots into predictions of what room I am in at any given time

#Arrays containing room values
bdrmsbools = [[]]
ktchnsbools = [[]]
lvngrmsbools = [[]]
wybool = [[]]

#Moving average filters smooth the data - this was found to give far better results
for i in range(1, len(bdrms)):
    bdrms[i] = np.convolve(bdrms[i], np.ones(5)/5, mode='valid')
    ktchns[i] = np.convolve(ktchns[i], np.ones(5)/5, mode='valid')
    lvngrms[i] = np.convolve(lvngrms[i], np.ones(5)/5, mode='valid')

#For every days worth of data
day = 0
for i in range(len(lvngrms)):

    #For every minutes woth of data
    for j in range(len(lvngrms[i])):
        bdrm = bdrms[i][j]
        lvngrm = lvngrms[i][j]
        ktchn = ktchns[i][j]

        #If out of the house
        if bdrm == -100 and ktchn == -100 and lvngrm == -100:
            wybool[day].append(1)
            ktchnsbools[day].append(0)
            bdrmsbools[day].append(0)
            lvngrmsbools[day].append(0)

        #if in the bedrrom
        elif bdrm >= ktchn and bdrm >= lvngrm:
            wybool[day].append(0)
            ktchnsbools[day].append(0)
            bdrmsbools[day].append(1)
            lvngrmsbools[day].append(0)

        #If in the kitchen
        elif ktchn >= bdrm and ktchn >= lvngrm:
            wybool[day].append(0)
            ktchnsbools[day].append(1)
            bdrmsbools[day].append(0)
            lvngrmsbools[day].append(0)

        #If in the livingroom
        elif lvngrm >= bdrm and lvngrm >= ktchn:
            wybool[day].append(0)
            ktchnsbools[day].append(0)
            bdrmsbools[day].append(0)
            lvngrmsbools[day].append(1)

        #If I wasn't in the house this day, remove from the dataset
        if sum(wybool[-1]) == 1440:
            print()
            wybool.pop()
            ktchnsbools.pop()
            bdrmsbools.pop()
            lvngrmsbools.pop()
            day-=1
    day+=1
    wybool.append([])
    ktchnsbools.append([])
    bdrmsbools.append([])
    lvngrmsbools.append([])


In [ ]:
#Plot my processed room locatiion for ewach dat i was iin the house

for i in range(1, len(wybool)):
    try:
        fig, ax = plt.subplots()
        ax.plot(times[i][:1300], bdrmsbools[i][:1300], linewidth=1.0, label = "Bedroom")
        ax.plot(times[i][:1300], lvngrmsbools[i][:1300], linewidth=1.0, label = "Living Room")
        ax.plot(times[i][:1300], ktchnsbools[i][:1300], linewidth=1.0, label = "Kitchen")
        
        every_nth = 240
        for n, label in enumerate(ax.xaxis.get_ticklabels()):
            if n % every_nth != 0:
                label.set_visible(False)

        plt.show()
    except:
        pass